## FFT playground
---

### load and filter

In [ ]:
# digits related code
from digits.data import matimport
from digits.data import select
from digits.transform.dimreduction import SubsampleTransform, FFTransform
from digits.utils import dotdict

from scipy import fftpack, signal

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [ ]:
imp = matimport.Importer(dataroot='../../data/thomas/artcorr/')
imp.open('3130.h5')
samples = imp.store.samples[:31]
targets = imp.store.targets[:31]

print(samples.shape)

samples, targets = select.fromsessionlist(samples, targets, ['1'])
samples, targets = select.frompresentationlist(samples, targets, list(range(10)))
samples = select.fromtimerange(samples, 201, 700)
samples = select.fromchannelblacklist(samples, ['LHEOG', 'RHEOG', 'IOL'])

subsample_width=1
samples = SubsampleTransform(width=subsample_width, verbose=True).transform(samples)

In [ ]:
flen=len(samples.iloc[0].loc['C1'])
print(flen)
#print(samples.iloc[0].loc['C1'].index.values)
print(select.getchannelnames(samples))

### Plots
---

In [ ]:
#samples2 = SubsampleTransform(width=2).transform(samples)
rcParams['figure.figsize'] = (8,2)
for c in ['C1', 'Fpz', 'O1', 'O2', 'Oz']:
    plt.plot(samples.iloc[0].loc[c].values, color='black')
    plt.title('Channel: '+c)
    plt.xlabel('t (ms)')
    plt.ylabel('Voltage')
    plt.show()

In [ ]:
samples2 = FFTransform(verbose=True, bins=40, multiplesof=2, fmin=2, fmax=150,
                       rate=subsample_width/1000).transform(samples)

In [ ]:
rcParams['figure.figsize'] = (8,2)
freqs = [int(x.split("_")[1]) for x in samples2.loc[:,'C1'].columns.values]
print(freqs)
for c in ['C1', 'Fpz', 'O1']:
    plt.bar(freqs, samples2.iloc[0].loc[c].values, color='black')
    plt.title('Channel: '+c)
    plt.xlabel('Power Spectrum (Hz)')
    plt.ylabel('Magnitude')
    plt.show()

In [ ]:
samples2 = FFTransform(verbose=True, bins=100, multiplesof=2, fmax=250,
                       rate=subsample_width/1000, power=False).transform(samples)

In [ ]:
rcParams['figure.figsize'] = (8,2)
freqs = [int(x.split("_")[1]) for x in samples2.loc[:,'C1'].columns.values]
for c in ['C1', 'Fpz']:
    plt.bar(freqs, samples2.iloc[0].loc[c].values)
    plt.title('Channel: '+c)
    plt.xlabel('Frequencies (Re/Im interleaved) (Hz)')
    plt.ylabel('Value')
    plt.show()

In [ ]:
x1 = samples2.iloc[0].loc['C1'].iloc[0:10]
x1

In [ ]:
x2 = samples2.iloc[0].loc['C1'].iloc[0:10]
x2

### Questions
---

**window**
 + hamming, hanning, blackman (default)?
 + tune?
 
**how to handle Re() and Im()**
 + currently: magnitude `sqrt(Re(x) + Im(x))`
 + drop Im() and then use abs() ?
 
**What to do with f=0 and f=max**
 
 ---

**freq resolution and averaging**
 + need to reduce dimensions
 + subsample and then cut? (loses high freq spectrum)
 + or cut and then average? (loses resolution)
 + average evenly spaced?

In [ ]:
print(fftpack.rfftfreq(500, 1/1000))

# drop 75%
print(fftpack.rfftfreq(125, 1/250))
print(fftpack.rfftfreq(500, 1/1000).reshape(-1,4).mean(axis=1))